In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
from xml.etree import ElementTree as ET
from concurrent.futures import ThreadPoolExecutor, wait, ALL_COMPLETED
from time import perf_counter
from queue import Queue

In [2]:
# idea from https://stackoverflow.com/a/76596818
def process_quakeml(responseContent):
    path = []
    rows = [] # no outer dict because some events don't have some tag (different columns -> rows don't match)
    parser = ET.XMLPullParser(events=("start","end"))
    parser.feed(responseContent)
    
    for event, element in parser.read_events():
        key = element.tag[element.tag.rfind("}")+1:] # gets rid of namespace
        exclude_tags = ["quakeml", "eventParameters", "event"] # useless info
        if event == "start" and key == "event":
            row = {} # function scope
            path.append(key)
        elif event == "end" and key == "event": # end of an earthquake event
            rows.append(row)
            path.pop()
        else:
            if event == "start" and key not in exclude_tags:
                path.append(key)
            elif event == "end" and key not in exclude_tags and "event" in path: # needs to be an event in path, no metadata
                current_path = "/".join(path)
                row[current_path] = element.text
                path.pop()    
    return rows

def get_quakeml(startTime, endTime, queue):
    dfList = []
    session = requests.Session()
    adapter = requests.adapters.HTTPAdapter(pool_maxsize=100) 
    session.mount('https://', adapter)
    parameters = {"format":"quakeml", "starttime":startTime, "endtime":endTime, "limit":20000, "minmagnitude":0, 'orderby':'time-asc', 'eventtype':'earthquake'}
    start = perf_counter()
    response = session.get("https://earthquake.usgs.gov/fdsnws/event/1/query", params=parameters)
    print(f"timer {perf_counter()-start} {queue.qsize()} {startTime}")
    if response.status_code == 204:
        pass
    elif response.status_code == 503: # too much, split
        timeList = split_time(parameters["starttime"], parameters["endtime"], timedelta(days=365/12), 3)
        for i in range(len(timeList)-1):
            queue.put((timeList[i], timeList[i+1]))
    else:
        rows = process_quakeml(response.content)
        print(startTime)
        datetime.strptime(rows[-1]["event/origin/time/value"], "%Y-%m-%dT%H:%M:%S.%fZ") # time of youngest row (earthquake): start of new query  
        dfList.append(pd.DataFrame(rows))
        if len(rows) == 20000 and parameters["starttime"] < parameters["endtime"]: # max records reached
            queue.put((datetime.strptime(rows[-1]["event/origin/time/value"], "%Y-%m-%dT%H:%M:%S.%fZ"), endTime))
    
    if len(dfList) == 0:
        return None
    else:
        return pd.concat(dfList, axis=0, ignore_index=True)

In [3]:
def process_json(features):
    rows = []        
    for index in range(len(features)):
        earthquake = features[index]
        prop = earthquake['properties']
        coor = earthquake['geometry']['coordinates']
        rows.append([earthquake['id'],coor[0],coor[1],coor[2],prop['mag'],prop['place'],prop['time'],prop['updated'],prop['tz'],prop['url'],
                     prop['detail'],prop['felt'],prop['cdi'],prop['mmi'],prop['alert'],prop['status'],prop['tsunami'],prop['sig'],prop['net'],
                     prop['code'],prop['ids'],prop['sources'],prop['types'],prop['nst'],prop['dmin'],prop['rms'],prop['gap'],prop['magType'],
                     prop['type'],prop['title']])
        
    return pd.DataFrame(rows, columns=['id','longitude','latitude','depth','mag','place','time','updated','tz','url','detail','felt','cdi',
                                                  'mmi','alert','status','tsunami','sig','net','code','ids','sources','types','nst','dmin','rms',
                                                  'gap','magType','title','type'])
def get_json(startTime, endTime):
    dfList = []
    session = requests.Session()
    adapter = requests.adapters.HTTPAdapter(pool_maxsize=250) 
    session.mount('https://', adapter)
    parameters = {"format":"geojson", "starttime":startTime, "endtime":endTime, "limit":20000, "minmagnitude":0, 'orderby':'time-asc', 'eventtype':'earthquake'}

    while parameters["starttime"] <= endTime:
        response = session.get("https://earthquake.usgs.gov/fdsnws/event/1/query", params=parameters)
        if response.status_code == 503:
            timeList = split_time(parameters["starttime"], endTime, timedelta(days=365/6), 6) 
                
            for i in range(len(timeList)-1): 
                parameters["starttime"] = timeList[i]
                parameters["endtime"] = timeList[i+1]
                response = session.get("https://earthquake.usgs.gov/fdsnws/event/1/query", params=parameters)
                features = response.json()['features']
                dfList.append(process_json(features))
            break # went through time list; end query
        else:
            features = response.json()['features']
            if len(features) == 0: # no results for query; combine dfList
                break
            parameters["starttime"] = datetime.utcfromtimestamp(features[-1]['properties']['time']/1000) # USGS used UTC timezone    
            dfList.append(process_json(features))
            if len(features) < 20000: # the query has reached its last earthquake
                break

    if len(dfList) == 0:
        return None
    else:
        return pd.concat(dfList, axis=0, ignore_index=True) 


In [ ]:
# taken from https://www.geeksforgeeks.org/python-divide-date-range-to-n-equal-duration/
# https://stackoverflow.com/a/29721341
def split_time(startTime, endTime, minDelta, divSegment):
    diff = endTime - startTime
    timeList = [startTime]
    while diff > minDelta: # arbitrary (split more recent data)
        segment = diff / divSegment # arbitrary division by 6
        diff = diff - segment 
        startTime = startTime + segment
        timeList.append(startTime)
    timeList.append(endTime)
    print("time %d" % len(timeList))
    return timeList
    
def get_data():
    start = perf_counter()
    startTime, endTime = datetime(1568, 1, 1, 0, 0, 0), datetime(2024, 6, 15) # 1568 get first record (quakeml)
    timeList = split_time(startTime, endTime, timedelta(days=365/12), 12)
    quakeList, jsonList, quakeFutures = [], [], []
    quakeQueue, jsonQueue = Queue(), Queue()

    for i in range(len(timeList)-1):
        quakeQueue.put((timeList[i],timeList[i+1]))
        jsonQueue.put((timeList[i],timeList[i+1]))

    while not quakeQueue.empty(): 
        print("not empty: %d" % quakeQueue.qsize())
        executor = ThreadPoolExecutor(max_workers=100)
        print("threadpool")
        for i in range(quakeQueue.qsize()):
            parameters = quakeQueue.get()
            quakeFutures.append(executor.submit(get_quakeml, parameters[0], parameters[1], quakeQueue))
            
            #jsonList = executor.map(get_json, timeList[:-1], timeList[1:])
        
    wait(quakeFutures, return_when=ALL_COMPLETED)

    quakeList = [future.result() for future in quakeFutures]
    quakeDf = pd.concat(quakeList, ignore_index=True)
    #jsonDf = pd.concat(jsonList, ignore_index=True)
    
    #print(len(quakeDf.index))
    #jsonDf.drop_duplicates(inplace=True) # dates do overlap although miniscule
    #quakeDf.drop_duplicates(inplace=True)
    #print(len(quakeDf.index))
    #jsonDf.reset_index(drop=True, inplace = True)
    #quakeDf.reset_index(inplace=True)
    print(f"timer {perf_counter()-start}")
    return quakeDf
    '''
    # These are all NaN's
    df.drop("felt", axis=1, inplace=True)
    df.drop("cdi", axis=1, inplace=True) # max intensity (dyfi)
    df.drop("mmi", axis=1, inplace=True) # max instrumental intensity (shakemap)
    df.drop("alert", axis=1, inplace=True) # not useful
    df.drop("type", axis=1, inplace=True) # redundant
    df.drop("place", axis=1, inplace=True) # no need to have a reference point when long and lat are provided
    df.dropna(inplace=Tp rowrue) # drop all rows with a missing value
    
    for i, r in df.iterrows(): # accurate reviewed data
        if r["status"] == "automatic" or r["status"] == "deleted":
            df.drop(index=i, inplace=True)
    df.drop("status", axis=1, inplace=True) # redundant
    df["time"] = pd.to_datetime(df["time"]) # convert object to datetime
    
    netlocmag_identical = True
    for i, r in df.iterrows(): # check for redundancy in columns
        if r["locationSource"] != r["magSource"] or r["locationSource"] != r["net"]:
            netlocmag_identical = False
    if netlocmag_identical: # rename the column to combine and drop the others
        df.rename(columns={"net": "netlocmagSource"}, inplace=True) 
        df.drop("magSource", axis=1, inplace=True)
        df.drop("locationSource", axis=1, inplace=True)'''
hi = get_data()

time 101
not empty: 100
threadpool
timer 0.6653590271016583 10 1568-01-01 00:00:00
1568-01-01 00:00:00
timer 1.1628396379528567 0 1702-03-02 16:05:45.833333
1702-03-02 16:05:45.833333
timer 1.4852371379965916 0 1900-09-12 15:25:52.054631
timer 1.4998168379534036 0 1815-11-11 15:43:22.073705
1815-11-11 15:43:22.073705
1900-09-12 15:25:52.054631
timer 1.7220964550506324 0 1606-01-14 02:00:00
1606-01-14 02:00:00
timer 1.7158757949946448 0 1889-06-12 10:17:18.605052
1889-06-12 10:17:18.605052
timer 1.838196607073769 0 1833-03-30 14:24:45.234230
1833-03-30 14:24:45.234230
timer 1.884523094049655 0 1729-01-09 02:45:17.013889
1729-01-09 02:45:17.013889
timer 1.9680766159435734 0 1863-10-14 08:16:38.009318
1863-10-14 08:16:38.009318
timer 2.057446461985819 0 1796-11-23 12:47:18.625860
1796-11-23 12:47:18.625860
timer 2.081022365950048 0 1640-11-26 09:50:00
1640-11-26 09:50:00
timer 2.191749162040651 0 1877-03-04 15:35:14.841875
1877-03-04 15:35:14.841875
timer 2.348964371951297 0 1911-01-05 10

In [ ]:
print(hi)